## Global Variable and Functions

In [20]:
import itertools
import seaborn as sns
import datetime
from collections import OrderedDict
import warnings
# warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import numpy.random as npr
from pylab import plt, mpl
import math
pd.set_option("display.max_columns", None)
sns.set_style("darkgrid")

T=3
FREQUENCY = 'Weekly'
Year1_Step = 52
Year2_Step = 52
Year3_Step = 51

## Make up a toy data set

In [21]:
Toy_df = pd.DataFrame(columns=['Year1_Beta','Year1_Sigma','Year2_Beta','Year2_Sigma','Year3_Beta','Year3_Sigma',
                               'Total_Beta','Total_Sigma'],data=npr.standard_exponential([5,8]))

Toy_df['Channel'] = ['Ch1', 'Ch2', 'Ch3', 'Ch4', 'Ch5']

In [22]:
Toy_df

,Year1_Beta,Year1_Sigma,Year2_Beta,Year2_Sigma,Year3_Beta,Year3_Sigma,Total_Beta,Total_Sigma,Channel
0,0.685994,0.903489,0.336807,2.301974,2.632105,0.841765,1.574976,0.225879,Ch1
1,1.223270,0.453877,0.800685,0.062940,2.278373,3.509709,0.456905,0.021946,Ch2
2,2.114214,0.206108,1.320888,0.145631,0.373330,0.505075,2.001086,1.211118,Ch3
3,1.466886,1.127267,0.040260,0.314592,0.081626,1.087048,0.358551,1.215615,Ch4
4,0.292049,1.336780,2.094373,0.997286,3.229589,0.085106,0.566373,0.144781,Ch5
